In [84]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [85]:
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from collections import defaultdict
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    N_REDUCED_SCNEARIOS,
    ALL_VEHICLE_TYPES,
    PATH_RESULTS_SUMMARY,
)
from modules.stochastic_program.factory import StochasticProgramFactory



### Prepare scenario data
Make scenarios complete cross product

In [86]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [87]:
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [88]:
complete_index = pd.MultiIndex.from_product(
    [
        scenarios.index.get_level_values('scenarios').unique(),
        pd.Index(hex_ids, name="start_hex_ids"),
        pd.Index(hex_ids, name="end_hex_ids"),
        scenarios.index.get_level_values('time').unique(),
        scenarios.index.get_level_values('vehicle_types').unique(),
    ]
)

In [89]:
scenarios = scenarios.reindex(complete_index, fill_value=0)

Find tree structure for non-anticipativity constraints

In [90]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [91]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [92]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

## Get Model Parameters


In [95]:
distances = pd.read_pickle(PATH_DISTANCES)

In [96]:
cost = defaultdict(lambda: defaultdict(dict))
profit = defaultdict(lambda: defaultdict(dict))

In [98]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [109]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [180]:
vehicle_types = ALL_VEHICLE_TYPES
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}
factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df, vehicle_types)
factory.set_initial_allocation(FLEET_CAPACITY)

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.02 seconds
_convert_demand finished in 0.14 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.16 seconds
_set_max_demand finished in 0.02 seconds
set_initial_allocation finished in 0.00 seconds


In [181]:
stochastic_program = factory.create_stochastic_program()

create_stochastic_program finished in 0.00 seconds


In [182]:
stochastic_program.relocations_disabled = False

In [183]:
stochastic_program.create_model()

_create_variables finished in 2.19 seconds
_create_objective_function finished in 1.67 seconds
_create_demand_constraints finished in 1.12 seconds
_create_relocation_binary_constraints finished in 0.01 seconds
_create_big_u_sum_constraints finished in 0.16 seconds
_create_unfulfilled_demand_binary_constraints finished in 0.01 seconds
_create_no_refused_demand_constraints finished in 0.01 seconds
_create_max_trips_constraints finished in 0.58 seconds
_create_vehicle_movement_constraints finished in 1.25 seconds
_create_initial_allocation_constraints finished in 0.00 seconds
_create_non_anticipativity_constraints finished in 0.26 seconds
_create_constraints finished in 3.41 seconds
create_model finished in 8.09 seconds


In [184]:
stochastic_program.solve(logPath='./log')

/home/moritz/miniconda3/envs/VR/lib/python3.8/site-packages/pulp/apis/coin_api.py:146: UserWarning: `logPath` argument replaces `msg=1`. The output will be redirected to the log file.
  warnings.warn('`logPath` argument replaces `msg=1`. The output will be redirected to the log file.')


Status: Optimal
Optimal Value of Objective Function:  4957.828195744558
Runtime without preprocessing: 5.34 seconds
solve finished in 8.14 seconds


# Benchmarks
## Different Capacities/ Disabled Relocations/ Value Of Perfect Information

In [23]:
capacities = [
    {
        "kick_scooter": 200,
        "bicycle": 100,
        "car": 50,
    },
    # {
    #     "kick_scooter": 1000,
    #     "bicycle": 500,
    #     "car": 250,
    # },{
    #     "kick_scooter": 2000,
    #     "bicycle": 1000,
    #     "car": 500,
    # }
]


Status: Optimal
Optimal Value of Objective Function:  3214.0597702779287
Runtime without preprocessing: 7.36 seconds
solve finished in 10.71 seconds


In [24]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)
factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.70 seconds
_convert_demand finished in 22.16 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 22.87 seconds
_set_max_demand finished in 0.34 seconds
Status: Optimal
Optimal Value of Objective Function:  4864.224455168022
Runtime without preprocessing: 23.37 seconds
solve finished in 35.58 seconds


Status: Optimal
Optimal Value of Objective Function:  4948.07548625501
Runtime without preprocessing: 20.10 seconds
solve finished in 32.99 seconds


Status: Optimal
Optimal Value of Objective Function:  -205.2693607114608
Runtime without preprocessing: 15.81 seconds
solve finished in 25.09 seconds


Status: Optimal
Optimal Value of Objective Function:  -164.90205545466662
Runtime without preprocessing: 13.91 seconds
solve finished in 21.44 seconds




In [25]:
results_df = pd.DataFrame.from_dict(results)

In [26]:
results_df

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,kick_scooter,bicycle,car,relocations_disabled,non_anticipativity_disabled
0,Optimal,4864.224455,23.374614,3563,189175,0,2037,200,100,50,False,False
1,Optimal,4948.075486,20.103675,3576,189172,0,2024,200,100,50,False,True
2,Optimal,-205.269361,15.812199,1847,193439,3753,0,200,100,50,True,False
3,Optimal,-164.902055,13.913911,1851,193434,3749,0,200,100,50,True,True


In [27]:
os.makedirs(os.path.dirname(PATH_RESULTS_SUMMARY), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SUMMARY)


## Single Modal Benchmark

In [28]:
last_demand = scenarios.copy()

last_demand_unstacked = last_demand.unstack("vehicle_types")
last_demand_unstacked[("demand", "car")] = (
    last_demand_unstacked[("demand", "kick_scooter")]
    + last_demand_unstacked[("demand", "car")]
    + last_demand_unstacked[("demand", "bicycle")]
)
last_demand = last_demand_unstacked.stack('vehicle_types')


In [29]:
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}

results = []

for vehicle_types in [["car"], ["bicycle"], ["kick_scooter"]]:
    current_vehicle_type = vehicle_types[0]
    current_fleet_capacity = {
        current_vehicle_type: FLEET_CAPACITY[current_vehicle_type]
    }

    last_demand.index = last_demand.index.set_levels(
        last_demand.index.get_level_values("vehicle_types").map(lambda x: current_vehicle_type),
        verify_integrity=False,
        level="vehicle_types",
    )

    factory = StochasticProgramFactory(
        last_demand,
        distances,
        probabilities,
        node_df,
        vehicle_types,
        include_methods=[None],
    )
    factory.set_initial_allocation(FLEET_CAPACITY)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    stochastic_program.create_model()
    stochastic_program.solve()
    # we transform the unfulfilled demand of the current lp into the demand for the next lp
    last_demand = stochastic_program.get_unfulfilled_demand().rename(
        columns={"unfulfilled_demand": "demand"}
    )

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": str(vehicle_types)}
    )


Status: Optimal
Optimal Value of Objective Function:  535.3306678123197
Runtime without preprocessing: 7.27 seconds
solve finished in 10.23 seconds
Status: Optimal
Optimal Value of Objective Function:  322.07286150242834
Runtime without preprocessing: 7.16 seconds
solve finished in 10.48 seconds
Status: Optimal
Optimal Value of Objective Function:  2587.3201908076935
Runtime without preprocessing: 7.36 seconds
solve finished in 10.46 seconds


In [30]:
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    ALL_VEHICLE_TYPES,
    include_methods=[None],
)
factory.set_initial_allocation(FLEET_CAPACITY)
stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]
stochastic_program.create_model()
stochastic_program.solve()

results.append(
    {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
)


In [31]:
results = pd.DataFrame.from_dict(results)
results

Status: Optimal


Optimal Value of Objective Function:  4864.224455168022
Runtime without preprocessing: 23.28 seconds
solve finished in 35.69 seconds


,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,535.330668,7.267965,492,48216,0,308,['car']
1,Optimal,322.072862,7.160683,960,47256,0,640,['bicycle']
2,Optimal,2587.320191,7.356493,1896,45360,0,1304,['kick_scooter']
3,Optimal,4864.224455,23.279787,3563,189175,0,2037,"[kick_scooter, bicycle, car]"


In [32]:
compare = results.iloc[[3]]
compare.append(results.iloc[range(3)].sum(), ignore_index=True)

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,4864.224455,23.279787,3563,189175,0,2037,"[kick_scooter, bicycle, car]"
1,OptimalOptimalOptimal,3444.723720,21.785141,3348,140832,0,2252,['car']['bicycle']['kick_scooter']


## Value Of The Stochastic Solution

In [33]:
demand = scenarios.copy()
demand = (
    demand.unstack("scenarios")["demand"]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0: "demand"})
)
demand["scenarios"] = 0
demand = demand.set_index("scenarios", append=True).reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)
demand['demand'] = demand['demand'] / N_REDUCED_SCNEARIOS
demand['floored'] = demand['demand'].apply(np.floor)
demand['ceiled'] = demand['demand'].apply(np.ceil)


In [34]:
results = []
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}

In [35]:
for rounding_mode in ["floored", "ceiled"]:
    factory = StochasticProgramFactory(
        demand[[rounding_mode]].rename(columns={rounding_mode: "demand"}),
        distances,
        probabilities,
        node_df,
        ALL_VEHICLE_TYPES,
        include_methods=[None],
    )
    factory.set_initial_allocation(FLEET_CAPACITY)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    
    # assign all weight to first scenario
    stochastic_program.weighting = {0: 1}

    # discard non-anticipativity constraints
    stochastic_program.non_anticipativity_disabled = True

    stochastic_program.create_model()
    stochastic_program.solve()

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
    )


Status: Optimal
Optimal Value of Objective Function:  4239.106750919575
Runtime without preprocessing: 2.35 seconds
solve finished in 3.68 seconds
Status: Optimal
Optimal Value of Objective Function:  5473.103517395035
Runtime without preprocessing: 2.22 seconds
solve finished in 3.70 seconds


In [36]:
results = pd.DataFrame.from_dict(results)

In [37]:
results

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,4239.106751,2.347817,429,22803,0,271,"[kick_scooter, bicycle, car]"
1,Optimal,5473.103517,2.223921,434,24940,0,266,"[kick_scooter, bicycle, car]"


In [38]:
scenarios.reset_index().nunique()

scenarios          8
start_hex_ids     82
end_hex_ids       82
time               2
vehicle_types      3
demand           109
dtype: int64